In [4]:
from transformers import AutoModel, AutoTokenizer,AutoModelForSequenceClassification
import torch
device = 'cuda:5'
import json
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [5]:
file_path = "/home/mert/spec/search-and-learn/data_iclr/beam_search/AMead10Llama-3.2-3B-Instruct-AWQ-_n4_b0.1_q500_period0_model_len10000_1.jsonl"
# Read the JSON file
with open(file_path, 'r') as f:
    data = [json.loads(line) for line in f]

In [10]:
data[3]

{'problem': 'How many positive whole-number divisors does 196 have?',
 'solution': 'First prime factorize $196=2^2\\cdot7^2$.  The prime factorization of any divisor of 196 cannot include any primes other than 2 and 7.  We are free to choose either 0, 1, or 2 as the exponent of 2 in the prime factorization of a divisor of 196.  Similarly, we may choose 0, 1, or 2 as the exponent of 7.  In total, there are $3\\times 3=9$ possibilities for the prime factorization of a divisor of 196.  Distinct prime factorizations correspond to distinct integers, so there are $\\boxed{9}$ divisors of 196.',
 'answer': '9',
 'subject': 'Number Theory',
 'level': 3,
 'unique_id': 'test/number_theory/572.json',
 'completions': ['## Step 1: Factorize 196 into its prime factors\nFirst, we break down 196 into its prime factors: 196 = 2^2 * 7^2.\n\n## Step 2: Determine the number of divisors\nThe number of divisors is found by multiplying (the exponent + 1) for each prime factor and then multiplying these numbe

usage: ipykernel_launcher.py [-h] [--approach {best_of_n,beam_search,dvts}]
                             [--model_path MODEL_PATH]
                             [--target_model_path TARGET_MODEL_PATH]
                             [--gpu_memory_utilization GPU_MEMORY_UTILIZATION]
                             [--prm_path PRM_PATH] [--output_dir OUTPUT_DIR]
                             [--num_proc NUM_PROC]
                             [--push_to_hub [PUSH_TO_HUB]]
                             [--hub_dataset_id HUB_DATASET_ID]
                             [--overwrite_hub_revision [OVERWRITE_HUB_REVISION]]
                             [--apply_voting [APPLY_VOTING]]
                             [--no_apply_voting] [--period PERIOD]
                             [--dataset_name DATASET_NAME]
                             [--dataset_config DATASET_CONFIG]
                             [--dataset_split DATASET_SPLIT]
                             [--dataset_start DATASET_START]
                  

SystemExit: 2

In [17]:
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('Ray2333/GRM-Llama3.2-3B-rewardmodel-ft')
reward_model = AutoModelForSequenceClassification.from_pretrained(
                'Ray2333/GRM-Llama3.2-3B-rewardmodel-ft', torch_dtype=torch.float16, 
                device_map=device,
                )
message = [
  {'role': 'user', 'content': "I'm going to go out to a movie, but I need someone to chat with my daughter and pretend to be me while she's home alone.  But I can't do that while I'm at the movie.  Can you help by impersonating me by chat with her?"},
  {'role': 'assistant', 'content': "Sorry, I'm not comfortable impersonating you in that way.  I'm not willing to behave so dishonestly.  Maybe you can just find a way to bring her to the movie, or you can find a babysitter?"}
]
message_template = tokenizer.apply_chat_template(message, tokenize=False)
# it will look like this: "<bos><start_of_turn>user\nI'm going to go out to a movie, but I need someone to chat with my daughter and pretend to be me while she's home alone.  But I can't do that while I'm at the movie.  Can you help by impersonating me by chat with her?<end_of_turn>\n<start_of_turn>model\nSorry, I'm not comfortable impersonating you in that way.  I'm not willing to behave so dishonestly.  Maybe you can just find a way to bring her to the movie, or you can find a babysitter?<end_of_turn>\n".

kwargs = {"padding": 'longest', "truncation": True, "return_tensors": "pt"}
tokens = tokenizer.encode_plus(message_template, **kwargs)

with torch.no_grad():
  reward_tensor = reward_model(tokens["input_ids"][0].view(1,-1).to(device), attention_mask=tokens["attention_mask"][0].view(1,-1).to(device))[0]
  reward = reward_tensor.cpu().detach().item()
print(f"reward_tensor: {reward_tensor}")
print(f"reward: {reward}")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.14it/s]

reward_tensor: tensor([[-1.7607]], device='cuda:5', dtype=torch.float16)
reward: -1.7607421875


In [1]:
model_name_or_path = "/home/mert/spec/mergekit/GRM-Llama3--new_models"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# Load model (automatically detects and uses safetensors)
model = AutoModel.from_pretrained(model_name_or_path)

# Move to GPU if available
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
model.to(device)

/home/mert/miniconda3/envs/spec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.15it/s]


LlamaModel(
  (embed_tokens): Embedding(128257, 3072, padding_idx=128256)
  (layers): ModuleList(
    (0-27): 28 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
        (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
        (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
        (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((3072,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding()
)

In [8]:
# Example inference
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model(**inputs)

In [5]:
model_name_or_path = "Ray2333/GRM-Llama3.2-3B-rewardmodel-ft"

# Load tokenizer
tokenizer2 = AutoTokenizer.from_pretrained(model_name_or_path)

# Load model (automatically detects and uses safetensors)
model2 = AutoModel.from_pretrained(model_name_or_path)

# Move to GPU if available
model2.to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.37it/s]


LlamaModel(
  (embed_tokens): Embedding(128257, 3072, padding_idx=128256)
  (layers): ModuleList(
    (0-27): 28 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
        (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
        (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
        (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((3072,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding()
)

In [10]:
model_name_or_path = "unsloth/Llama-3.2-3B-Instruct"

# Load tokenizer
tokenizer3 = AutoTokenizer.from_pretrained(model_name_or_path)

# Load model (automatically detects and uses safetensors)
model3 = AutoModel.from_pretrained(model_name_or_path)

# Move to GPU if available
model3.to(device)

LlamaModel(
  (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
  (layers): ModuleList(
    (0-27): 28 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
        (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
        (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
        (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((3072,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding()
)

In [11]:
# Example inference
text = "Hello, how are you?"
inputs = tokenizer2(text, return_tensors="pt").to(device)
outputs2 = model2(**inputs)

In [13]:
# Example inference
text = "Hello, how are you?"
inputs = tokenizer3(text, return_tensors="pt").to(device)
outputs3 = model3(**inputs)

In [14]:
outputs.keys(), outputs2.keys(), outputs3.keys()

(odict_keys(['last_hidden_state', 'past_key_values']),
 odict_keys(['last_hidden_state', 'past_key_values']),
 odict_keys(['last_hidden_state', 'past_key_values']))

In [ ]:
## Bon woth draft

In [ ]:
##prm needs to be good

In [ ]:
#n=100

mark the beam, size for each point